In [1]:
using BIPs

In [2]:
temporal_data_path = "../DataLake/raw/train.h5"

"../DataLake/raw/train.h5"

In [3]:
train_jets, labels = BIPs.read_data("TQ", temporal_data_path)
labels = [reinterpret(Bool, b == 1.0) for b in labels]
length(train_jets)

1210997

In [4]:
train_transf_jets = data2hyp(train_jets)
println("Transformed jets: ", length(train_transf_jets))

Transformed jets: 1210997


In [5]:
f_bip, specs = build_ip(order=4,
    levels=7)
    
function bip_data(dataset_jets)
    storage = zeros(length(dataset_jets), length(specs))
    for i = 1:length(dataset_jets)
        storage[i, :] = f_bip(dataset_jets[i])
    end
    storage
end

bip_data (generic function with 1 method)

In [6]:
embedded_jets = bip_data(train_transf_jets)
print("Embedded jets: ", sizeof(embedded_jets))

Embedded jets: 14280076624

Now, lets fit a simple model to the data.


In [7]:
using PyCall
@pyimport sklearn.ensemble as sk_ensemble
@pyimport sklearn.metrics as sk_metrics
@pyimport sklearn.model_selection as sk_model_selection

In [8]:
GBT = sk_ensemble.HistGradientBoostingClassifier(verbose=true).fit(embedded_jets, labels)

Binning 12.852 GB of training data: 44.647 s
Binning 1.428 GB of validation data: 0.737 s
Fitting gradient boosted rounds:
[1/100] 1 tree, 31 leaves, max depth = 8, train loss: 0.62489, val loss: 0.62511, in 0.409s
[2/100] 1 tree, 31 leaves, max depth = 9, train loss: 0.56882, val loss: 0.56918, in 0.420s
[3/100] 1 tree, 31 leaves, max depth = 8, train loss: 0.52185, val loss: 0.52234, in 0.357s
[4/100] 1 tree, 31 leaves, max depth = 10, train loss: 0.48195, val loss: 0.48252, in 0.364s
[5/100] 1 tree, 31 leaves, max depth = 9, train loss: 0.44813, val loss: 0.44878, in 0.533s
[6/100] 1 tree, 31 leaves, max depth = 9, train loss: 0.41905, val loss: 0.41980, in 0.519s
[7/100] 1 tree, 31 leaves, max depth = 10, train loss: 0.39385, val loss: 0.39469, in 0.361s
[8/100] 1 tree, 31 leaves, max depth = 9, train loss: 0.37207, val loss: 0.37298, in 0.370s
[9/100] 1 tree, 31 leaves, max depth = 10, train loss: 0.35289, val loss: 0.35387, in 0.403s
[10/100] 1 tree, 31 leaves, max depth = 9, tra

PyObject HistGradientBoostingClassifier(verbose=True)

In [9]:
GBT.score(embedded_jets, labels)

0.9253672800180347

In [10]:
train_sizes, train_scores, test_scores, fit_times, _ = sk_model_selection.learning_curve(
                                GBT,
                                embedded_jets,
                                labels,
                                cv=sk_model_selection.ShuffleSplit(n_splits=10, test_size=0.2),
                                n_jobs = -1)

# Lest test how we do performance

In [ ]:
test_data = "../DataLake/raw/test.h5"

In [ ]:
test_jets, test_labels = BIPs.read_data("TQ", temporal_data_path)
test_labels = [reinterpret(Bool, b == 1.0) for b in test_labels]
test_transf_jets = data2hyp(test_jets)
test_embedded_jets = bip_data(test_transf_jets)

In [ ]:
test_predictions = GBT.predict(test_embedded_jets)